In [0]:
service_credential = dbutils.secrets.get(scope="kv-databricks-scope",key="databricks-cyrtdata-sp")

spark.conf.set("fs.azure.account.auth.type.databricks-cyrtdata-sp.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.databricks-cyrtdata-sp.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.databricks-cyrtdata-sp.dfs.core.windows.net", "fe9a0284-f087-4ff3-accf-4da08b067df1")
spark.conf.set("fs.azure.account.oauth2.client.secret.databricks-cyrtdata-sp.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.databricks-cyrtdata-sp.dfs.core.windows.net", "https://login.microsoftonline.com/d9bad1c3-3b62-413e-9a30-3cf800e6a5ee/oauth2/token")

In [0]:
acct = "cyrtdata"  # STORAGE ACCOUNT name, not the app name

spark.conf.set(f"fs.azure.account.auth.type.{acct}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{acct}.dfs.core.windows.net",
               "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{acct}.dfs.core.windows.net",
               "fe9a0284-f087-4ff3-accf-4da08b067df1")   # your SP (app) client ID
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{acct}.dfs.core.windows.net",
               dbutils.secrets.get("kv-databricks-scope", "databricks-cyrtdata-sp"))
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{acct}.dfs.core.windows.net",
               "https://login.microsoftonline.com/d9bad1c3-3b62-413e-9a30-3cf800e6a5ee/oauth2/token")

root = "abfss://weather@cyrtdata.dfs.core.windows.net/"
dbutils.fs.mkdirs(root + "bronze/")
display(dbutils.fs.ls(root))


In [0]:
df = spark.createDataFrame([("ok", 1)])
df.write.mode("overwrite").parquet(root + "bronze/smoke_test/part=1/")
spark.read.parquet(root + "bronze/smoke_test/part=1/").show()

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS weather_bronze;

CREATE TABLE weather_bronze.ecmwf_file_index (
  run_date DATE,
  run_hour INT,
  path STRING,
  bytes BIGINT,
  ingested_at TIMESTAMP
) USING DELTA
LOCATION 'abfss://weather@cyrtdata.dfs.core.windows.net/bronze/ecmwf/_index';